In [57]:
import pandas as pd

In [58]:
nyc = pd.read_csv("../data/nyc_trees.csv")

sf = pd.read_csv("../data/sf_trees.csv")


/var/folders/5f/w2x0tg553jq8gh_60lnq6vr00000gn/T/ipykernel_36241/49643019.py:3: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  sf = pd.read_csv("../data/sf_trees.csv")


In [59]:
print(nyc.columns)
print(sf.columns)

Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward',
       'guards', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other', 'address', 'postcode',
       'zip_city', 'community board', 'borocode', 'borough', 'cncldist',
       'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct', 'state',
       'latitude', 'longitude', 'x_sp', 'y_sp', 'council district',
       'census tract', 'bin', 'bbl'],
      dtype='object')
Index(['TreeID', 'qLegalStatus', 'qSpecies', 'qAddress', 'SiteOrder',
       'qSiteInfo', 'PlantType', 'qCaretaker', 'qCareAssistant', 'PlantDate',
       'DBH', 'PlotSize', 'PermitNotes', 'XCoord', 'YCoord', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')


In [60]:
nyc_ren = nyc.rename(columns={
    "spc_common": "species",
    "tree_dbh": "dbh",
})

nyc_ren["city"] = "NYC"

sf_ren = sf.rename(columns={
    "qSpecies": "species",
    "DBH": "dbh"
})

sf_ren["city"] = "SF"


In [61]:
cols = ["species", "dbh", "city"]
nyc_ren = nyc_ren[cols]
sf_ren  = sf_ren[cols]

In [62]:
def clean_dbh(df):
    df = df.copy()
    df["dbh"] = pd.to_numeric(df["dbh"], errors="coerce")
    df = df[df["dbh"].notna()]
    df = df[df["dbh"] > 0]
    return df

nyc_ren = clean_dbh(nyc_ren)
sf_ren  = clean_dbh(sf_ren)

In [63]:
trees = pd.concat([nyc_ren, sf_ren], ignore_index=True)

In [64]:
def size_class(dbh):
    if dbh < 10:
        return "small"
    elif dbh < 20:
        return "medium"
    else:
        return "large"

trees["size_class"] = trees["dbh"].apply(size_class)

In [65]:
size_counts = (
    trees
    .groupby(["city", "size_class"])
    .size()
    .reset_index(name="count")
)


In [66]:
total_by_city = (
    trees
    .groupby("city")
    .size()
    .reset_index(name="total_trees")
)

In [67]:
size_summary = pd.merge(
    size_counts,
    total_by_city,
    on="city",
    how="left"
)

In [68]:
size_summary["percent"] = size_summary["count"] / size_summary["total_trees"] * 100

In [69]:
print(size_summary)

  city size_class   count  total_trees    percent
0  NYC      large  112544       665856  16.902153
1  NYC     medium  224856       665856  33.769464
2  NYC      small  328456       665856  49.328383
3   SF      large   13616       151781   8.970820
4   SF     medium   45251       151781  29.813349
5   SF      small   92914       151781  61.215831


In [72]:
size_summary.to_csv("../output/species_compare2.csv", index=False)